# 1.generate  dataset
# 2.Train the classifier
# 3.detect the face

In [12]:
import cv2
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.3,5)   
        #scaling fctor =1.3
        #Minimum neighbor =5

        if faces is ():
            return None
        for(x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face


    cap = cv2.VideoCapture(0)
    id =1
    img_id=0
    
    while True:
        ret,frame=cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame),(300,300))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path,face)
            cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            #(50,50) is the origin point from where text id to be written
            #font scale = 1
            #thickness = 2

            cv2.imshow("Cropped face",face)
            if cv2.waitKey(1)==13 or int(img_id)==100:
                #13 for enter key or escape 
                break
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting saamples is completed......")   
    

generate_dataset()

<>:10: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
C:\Users\lenovo\AppData\Local\Temp\ipykernel_4700\307335961.py:10: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():


# Train the classifier and save it

In [11]:
import numpy as np
from PIL import Image
import os
import cv2

def train_classifier(data_dir):
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')     #convert to gray image
        imageNp = np.array(img,'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)

    # train the classifiers and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")

train_classifier("data")

# detect the face and named it if it is already stored in our dataset

In [4]:
import cv2
import numpy as np
from PIL import Image
import os

def draw_boundry(img,classifier,scaleFactor,minNeighbors,color,text,clf):
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image,scaleFactor,minNeighbors)

    coords = []

    for(x,y,w,h) in features:
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        id,pred = clf.predict(gray_image[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))

        if confidence>77:
            if id==1:
                cv2.putText(img,"Khalil",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
            if id==2:
                cv2.putText(img,"shaikh",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
        else:
            cv2.putText(img,"UNKNOWN",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)

        coords= [x,y,w,h]
    return coords

def recognize(img,clf,faceCascade):
    coords =draw_boundry(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
    return img

faceCascade =  cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret,img = video_capture.read()
    img = recognize(img,clf,faceCascade)
    cv2.imshow("face detection",img)

    if cv2.waitKey(1) == 13:
        break

video_capture.release()
cv2.destroyAllWindows()